In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.keras import optimizers, losses, activations, models
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.python.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, Flatten
from sklearn.metrics import f1_score, accuracy_score


In [13]:
df_train = pd.read_csv("C:/00ETH/ml4h/Project1/archive/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("C:/00ETH/ml4h/Project1/archive/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


In [14]:
def identity_block(x,filter):
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)  
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 3
    x = tf.keras.layers.Conv1D(filter[2], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[2], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[2], kernel_size=3, padding = 'same')(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [15]:
def identity_block2(x,filter):
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)  
    x = tf.keras.layers.Activation('relu')(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [16]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [17]:
def convolutional_block2(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)      
    # Layer 3
    x = tf.keras.layers.Conv1D(filter[2], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[2], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[2], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)         
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x_skip)
    x_skip = tf.keras.layers.Conv1D(filter[2], kernel_size=3, padding = 'same')(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [18]:
def get_model1():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = identity_block(img_1,[16,32,16])
    img_shortcut = convolutional_block(img_shortcut,[32,64])
    img_shortcut = identity_block2(img_shortcut,[64,64])
    img_shortcut = convolutional_block2(img_shortcut,[32,32,64])
    img_shortcut = convolutional_block(img_shortcut,[64,128])
    img_shortcut = identity_block2(img_shortcut,[64,128])
    img_shortcut = convolutional_block(img_shortcut,[64,64])
    img_shortcut = convolutional_block(img_shortcut,[32,32])
    img_shortcut = identity_block2(img_shortcut,[32,32])
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = Flatten()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [19]:
model1 = get_model1()
file_path = "Resnet_cnn_mitbih1.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_87 (Conv1D)              (None, 187, 16)      96          input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_88 (Conv1D)              (None, 187, 16)      1296        conv1d_87[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, 93, 16)       0           conv1d_88[0][0]                  
____________________________________________________________________________________________

In [20]:
model1.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model1.load_weights(file_path)

pred_test = model1.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.95626, saving model to Resnet_cnn_mitbih1.h5
78798/78798 - 401s - loss: 0.2437 - acc: 0.9349 - val_loss: 0.1540 - val_acc: 0.9563
Epoch 2/1000

Epoch 00002: val_acc improved from 0.95626 to 0.95763, saving model to Resnet_cnn_mitbih1.h5
78798/78798 - 347s - loss: 0.1351 - acc: 0.9624 - val_loss: 0.1403 - val_acc: 0.9576
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.95763
78798/78798 - 340s - loss: 1.0291 - acc: 0.8427 - val_loss: 0.6563 - val_acc: 0.8285
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.95763
78798/78798 - 339s - loss: 0.6575 - acc: 0.8276 - val_loss: 0.6564 - val_acc: 0.8285
Epoch 5/1000

Epoch 00005: val_acc did not improve from 0.95763

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
78798/78798 - 339s - loss: 0.6576 - acc: 0.8276 - val_loss: 0.6562 - val_acc: 0.8285
Epoch 6/1000

Epoch 00006: val_acc did no

In [21]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.7550709972072144 
Test accuracy score : 0.9570162616480906 


In [22]:
def get_model2():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = convolutional_block(img_1,[16,32])
    img_shortcut = identity_block2(img_shortcut,[32,32])
    img_shortcut = identity_block2(img_shortcut,[32,32])   
    img_shortcut = convolutional_block(img_shortcut,[32,64])
    img_shortcut = convolutional_block(img_shortcut,[64,128])
    img_shortcut = identity_block2(img_shortcut,[64,128]) 
    img_shortcut = convolutional_block(img_shortcut,[64,32])
    img_shortcut = identity_block2(img_shortcut,[32,32])
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [23]:
model2 = get_model2()
file_path = "Resnet_cnn_mitbih2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_148 (Conv1D)             (None, 187, 16)      96          input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_149 (Conv1D)             (None, 187, 16)      1296        conv1d_148[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_3 (MaxPooling1D)  (None, 93, 16)       0           conv1d_149[0][0]                 
____________________________________________________________________________________________

In [24]:
model2.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model2.load_weights(file_path)

pred_test2 = model2.predict(X_test)
pred_test2 = np.argmax(pred_test2, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.93833, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 304s - loss: 0.2696 - acc: 0.9270 - val_loss: 0.1988 - val_acc: 0.9383
Epoch 2/1000

Epoch 00002: val_acc improved from 0.93833 to 0.95946, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 321s - loss: 0.1701 - acc: 0.9537 - val_loss: 0.1409 - val_acc: 0.9595
Epoch 3/1000

Epoch 00003: val_acc improved from 0.95946 to 0.97168, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 318s - loss: 0.1427 - acc: 0.9609 - val_loss: 0.1096 - val_acc: 0.9717
Epoch 4/1000

Epoch 00004: val_acc improved from 0.97168 to 0.97225, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 319s - loss: 0.1243 - acc: 0.9661 - val_loss: 0.1044 - val_acc: 0.9722
Epoch 5/1000

Epoch 00005: val_acc improved from 0.97225 to 0.97465, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 318s - loss: 0.1087 - acc: 0.9709 - val_loss: 0.0947 - val_acc: 0

In [25]:
f1 = f1_score(Y_test, pred_test2, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test2)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.918855920551451 
Test accuracy score : 0.9853371094463731 


In [26]:
def get_model3():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = convolutional_block(img_1,[16,32])
    img_shortcut = identity_block2(img_shortcut,[32,32])
    img_shortcut = convolutional_block(img_shortcut,[32,64])
    img_shortcut = identity_block2(img_shortcut,[64,64]) 
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [27]:
model3 = get_model3()
file_path = "Resnet_cnn_mitbih3.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_196 (Conv1D)             (None, 187, 16)      96          input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_197 (Conv1D)             (None, 187, 16)      1296        conv1d_196[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_4 (MaxPooling1D)  (None, 93, 16)       0           conv1d_197[0][0]                 
____________________________________________________________________________________________

In [28]:
model3.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model3.load_weights(file_path)

pred_test3 = model3.predict(X_test)
pred_test3 = np.argmax(pred_test3, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.95592, saving model to Resnet_cnn_mitbih3.h5
78798/78798 - 2463s - loss: 0.2428 - acc: 0.9342 - val_loss: 0.1603 - val_acc: 0.9559
Epoch 2/1000

Epoch 00002: val_acc did not improve from 0.95592
78798/78798 - 191s - loss: 0.1569 - acc: 0.9577 - val_loss: 0.2001 - val_acc: 0.9434
Epoch 3/1000

Epoch 00003: val_acc improved from 0.95592 to 0.96859, saving model to Resnet_cnn_mitbih3.h5
78798/78798 - 189s - loss: 0.1327 - acc: 0.9639 - val_loss: 0.1152 - val_acc: 0.9686
Epoch 4/1000

Epoch 00004: val_acc improved from 0.96859 to 0.97145, saving model to Resnet_cnn_mitbih3.h5
78798/78798 - 189s - loss: 0.1192 - acc: 0.9674 - val_loss: 0.1063 - val_acc: 0.9714
Epoch 5/1000

Epoch 00005: val_acc improved from 0.97145 to 0.97396, saving model to Resnet_cnn_mitbih3.h5
78798/78798 - 190s - loss: 0.1104 - acc: 0.9703 - val_loss: 0.0954 - val_acc: 0.9740
Epoch 6/1000

Epoch 00006: val_acc i

In [29]:
f1 = f1_score(Y_test, pred_test3, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test3)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9056616752219036 
Test accuracy score : 0.9830988488945733 


In [30]:
def get_model4():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = convolutional_block2(img_1,[16,32,32])
    img_shortcut = identity_block(img_shortcut,[16,32,32])
    img_shortcut = convolutional_block2(img_shortcut,[32,32,64])
    img_shortcut = identity_block(img_shortcut,[32,64,64]) 
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = Flatten()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [31]:
model4 = get_model4()
file_path = "Resnet_cnn_mitbih4.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_222 (Conv1D)             (None, 187, 16)      96          input_6[0][0]                    
__________________________________________________________________________________________________
conv1d_223 (Conv1D)             (None, 187, 16)      1296        conv1d_222[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_5 (MaxPooling1D)  (None, 93, 16)       0           conv1d_223[0][0]                 
____________________________________________________________________________________________

In [32]:
model4.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model4.load_weights(file_path)

pred_test4 = model4.predict(X_test)
pred_test4 = np.argmax(pred_test4, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.96414, saving model to Resnet_cnn_mitbih4.h5
78798/78798 - 272s - loss: 0.1914 - acc: 0.9471 - val_loss: 0.1211 - val_acc: 0.9641
Epoch 2/1000

Epoch 00002: val_acc improved from 0.96414 to 0.96768, saving model to Resnet_cnn_mitbih4.h5
78798/78798 - 261s - loss: 0.1092 - acc: 0.9693 - val_loss: 0.1149 - val_acc: 0.9677
Epoch 3/1000

Epoch 00003: val_acc improved from 0.96768 to 0.97739, saving model to Resnet_cnn_mitbih4.h5
78798/78798 - 262s - loss: 0.0886 - acc: 0.9747 - val_loss: 0.0840 - val_acc: 0.9774
Epoch 4/1000

Epoch 00004: val_acc improved from 0.97739 to 0.98196, saving model to Resnet_cnn_mitbih4.h5
78798/78798 - 262s - loss: 0.0737 - acc: 0.9783 - val_loss: 0.0650 - val_acc: 0.9820
Epoch 5/1000

Epoch 00005: val_acc did not improve from 0.98196
78798/78798 - 262s - loss: 0.0659 - acc: 0.9807 - val_loss: 0.0726 - val_acc: 0.9801
Epoch 6/1000

Epoch 00006: val_acc di

In [33]:
f1 = f1_score(Y_test, pred_test4, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test4)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9099342110245219 
Test accuracy score : 0.9836469943358305 
